In [1]:
# Dependencies
import pandas as pd
import csv
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [2]:
breed_label = pd.read_csv("Resources/BreedLabels.csv")
breed_label

,BreedID,Type,BreedName
0,1,1,Affenpinscher
1,2,1,Afghan Hound
2,3,1,Airedale Terrier
3,4,1,Akbash
4,5,1,Akita
...,...,...,...
302,302,2,Torbie
303,303,2,Tortoiseshell
304,304,2,Turkish Angora
305,305,2,Turkish Van


In [3]:
submission = pd.read_csv("Resources/submission.csv")
submission

,PetID,AdoptionSpeed
0,e2dfc2935,3
1,f153b465f,2
2,3c90f3f54,1
3,e02abc8a3,4
4,09f0df7d1,3
...,...,...
3967,ae57f8d52,4
3968,83432904d,4
3969,399013029,4
3970,fd80b8c80,4


In [4]:
data = pd.read_csv("Resources/test.csv")
data = data[["PetID", "Type", "Age", "Breed1", "Breed2", "Gender","Health"]]
data

,PetID,Type,Age,Breed1,Breed2,Gender,Health
0,e2dfc2935,2,8,266,266,1,1
1,f153b465f,2,36,285,264,2,2
2,3c90f3f54,2,2,265,0,1,1
3,e02abc8a3,1,12,307,0,2,1
4,09f0df7d1,2,12,265,0,1,1
...,...,...,...,...,...,...,...
3967,ae57f8d52,1,5,307,307,1,1
3968,83432904d,1,30,307,307,1,1
3969,399013029,2,5,300,0,3,1
3970,fd80b8c80,1,24,307,0,2,1


### Create new data with select columns

In [5]:
merged = pd.merge(submission, data, on= "PetID")
merged

,PetID,AdoptionSpeed,Type,Age,Breed1,Breed2,Gender,Health
0,e2dfc2935,3,2,8,266,266,1,1
1,f153b465f,2,2,36,285,264,2,2
2,3c90f3f54,1,2,2,265,0,1,1
3,e02abc8a3,4,1,12,307,0,2,1
4,09f0df7d1,3,2,12,265,0,1,1
...,...,...,...,...,...,...,...,...
3967,ae57f8d52,4,1,5,307,307,1,1
3968,83432904d,4,1,30,307,307,1,1
3969,399013029,4,2,5,300,0,3,1
3970,fd80b8c80,4,1,24,307,0,2,1


In [6]:
merged["Type"]= merged["Type"].replace({1:'Dog',2:'Cat'})
merged["Gender"]= merged["Gender"].replace({1:'Male',2:'Female',3:'Not Specified'})
merged["Health"]= merged["Health"].replace({1:'Healthy',2:'Minor Injury',3:'Major Injury',0:"Not Specified"})
merged= merged.rename(columns={"Age": "ageinmonths", "Breed1": "BreedID"})
merged

,PetID,AdoptionSpeed,Type,ageinmonths,BreedID,Breed2,Gender,Health
0,e2dfc2935,3,Cat,8,266,266,Male,Healthy
1,f153b465f,2,Cat,36,285,264,Female,Minor Injury
2,3c90f3f54,1,Cat,2,265,0,Male,Healthy
3,e02abc8a3,4,Dog,12,307,0,Female,Healthy
4,09f0df7d1,3,Cat,12,265,0,Male,Healthy
...,...,...,...,...,...,...,...,...
3967,ae57f8d52,4,Dog,5,307,307,Male,Healthy
3968,83432904d,4,Dog,30,307,307,Male,Healthy
3969,399013029,4,Cat,5,300,0,Not Specified,Healthy
3970,fd80b8c80,4,Dog,24,307,0,Female,Healthy


In [7]:
merged_breed = pd.merge(merged, breed_label, on="BreedID")
merged_breed = merged_breed.drop(columns={"Type_y", "BreedID", "Breed2"})
merged_breed = merged_breed.rename(columns={"Type_x": "Type", "BreedName": "MainBreed"})
merged_breed= merged_breed.rename(columns={"PetID": "petid", "AdoptionSpeed": "adoptionspeed", "Type": "type", "Gender": "gender", "Health": "health", "MainBreed": "mainbreed"})

merged_breed

,petid,adoptionspeed,type,ageinmonths,gender,health,mainbreed
0,e2dfc2935,3,Cat,8,Male,Healthy,Domestic Short Hair
1,2191e428d,4,Cat,10,Female,Healthy,Domestic Short Hair
2,0ecec0922,4,Cat,7,Female,Healthy,Domestic Short Hair
3,23b3f4aaf,3,Cat,15,Female,Healthy,Domestic Short Hair
4,e46543f14,4,Cat,36,Female,Healthy,Domestic Short Hair
...,...,...,...,...,...,...,...
3967,1defc9276,4,Dog,34,Male,Minor Injury,Shar Pei
3968,8c23d699b,2,Dog,84,Male,Healthy,Afghan Hound
3969,b05179874,4,Cat,24,Male,Healthy,Snowshoe
3970,7c30bce68,1,Dog,5,Male,Healthy,Entlebucher


### Clean DataFrame

In [8]:
combined_data = merged_breed
combined_data["adoptionspeed"]= combined_data["adoptionspeed"].replace({0:'Same Day',1:'Within 1 Week',2:'Within 1 Month',3:'Within 2-3 Months',4:'Not Adopted, Probably Euthanized'})
combined_data

,petid,adoptionspeed,type,ageinmonths,gender,health,mainbreed
0,e2dfc2935,Within 2-3 Months,Cat,8,Male,Healthy,Domestic Short Hair
1,2191e428d,"Not Adopted, Probably Euthanized",Cat,10,Female,Healthy,Domestic Short Hair
2,0ecec0922,"Not Adopted, Probably Euthanized",Cat,7,Female,Healthy,Domestic Short Hair
3,23b3f4aaf,Within 2-3 Months,Cat,15,Female,Healthy,Domestic Short Hair
4,e46543f14,"Not Adopted, Probably Euthanized",Cat,36,Female,Healthy,Domestic Short Hair
...,...,...,...,...,...,...,...
3967,1defc9276,"Not Adopted, Probably Euthanized",Dog,34,Male,Minor Injury,Shar Pei
3968,8c23d699b,Within 1 Month,Dog,84,Male,Healthy,Afghan Hound
3969,b05179874,"Not Adopted, Probably Euthanized",Cat,24,Male,Healthy,Snowshoe
3970,7c30bce68,Within 1 Week,Dog,5,Male,Healthy,Entlebucher


In [9]:
dogs = merged_breed.loc[(merged_breed["type"] == "Dog")]

dogs

,petid,adoptionspeed,type,ageinmonths,gender,health,mainbreed
1469,e02abc8a3,"Not Adopted, Probably Euthanized",Dog,12,Female,Healthy,Mixed Breed
1470,0113cedff,"Not Adopted, Probably Euthanized",Dog,24,Female,Healthy,Mixed Breed
1471,0070b950a,"Not Adopted, Probably Euthanized",Dog,12,Female,Healthy,Mixed Breed
1472,be85036be,"Not Adopted, Probably Euthanized",Dog,5,Male,Healthy,Mixed Breed
1473,6a968e033,"Not Adopted, Probably Euthanized",Dog,6,Not Specified,Healthy,Mixed Breed
...,...,...,...,...,...,...,...
3966,ade5e46bc,Within 2-3 Months,Dog,12,Female,Healthy,Shar Pei
3967,1defc9276,"Not Adopted, Probably Euthanized",Dog,34,Male,Minor Injury,Shar Pei
3968,8c23d699b,Within 1 Month,Dog,84,Male,Healthy,Afghan Hound
3970,7c30bce68,Within 1 Week,Dog,5,Male,Healthy,Entlebucher


In [10]:
cats = merged_breed.loc[(merged_breed["type"] == "Cat")]
cats

,petid,adoptionspeed,type,ageinmonths,gender,health,mainbreed
0,e2dfc2935,Within 2-3 Months,Cat,8,Male,Healthy,Domestic Short Hair
1,2191e428d,"Not Adopted, Probably Euthanized",Cat,10,Female,Healthy,Domestic Short Hair
2,0ecec0922,"Not Adopted, Probably Euthanized",Cat,7,Female,Healthy,Domestic Short Hair
3,23b3f4aaf,Within 2-3 Months,Cat,15,Female,Healthy,Domestic Short Hair
4,e46543f14,"Not Adopted, Probably Euthanized",Cat,36,Female,Healthy,Domestic Short Hair
...,...,...,...,...,...,...,...
3958,88e23d011,Within 2-3 Months,Cat,7,Female,Healthy,Devon Rex
3960,b62b7f5b8,"Not Adopted, Probably Euthanized",Cat,14,Not Specified,Healthy,Japanese Bobtail
3963,6905c3150,"Not Adopted, Probably Euthanized",Cat,2,Not Specified,Healthy,Manx
3964,08e585e1a,"Not Adopted, Probably Euthanized",Cat,2,Not Specified,Healthy,Manx


In [11]:
dog_health = dogs.groupby(["gender", "health"]).nunique()
dog_health = dog_health[["petid"]]
dog_health = dog_health.rename(columns={"petid": "number_of_dogs"})
dog_health1 = dog_health.reset_index()
dog_health

number_of_dogs
gender        health                      
Female        Healthy                 1011
              Major Injury               2
              Minor Injury              36
Male          Healthy                  780
              Major Injury               3
              Minor Injury              40
Not Specified Healthy                  220
              Minor Injury               8

In [12]:
dog_age = dogs.groupby("adoptionspeed").mean()
dog_age = dog_age.reset_index()
dog_age["adoptionspeed"]= dog_age["adoptionspeed"].replace({0:'Same Day',1:'Within 1 Week',2:'Within 1 Month',3:'Within 2-3 Months',4:'Not Adopted, Probably Euthanized'})
dog_age

,adoptionspeed,ageinmonths
0,"Not Adopted, Probably Euthanized",16.800752
1,Same Day,14.040816
2,Within 1 Month,10.813187
3,Within 1 Week,16.563953
4,Within 2-3 Months,10.928375


In [13]:
dog_age_bin = dogs[["petid","ageinmonths"]]
bins = [0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144, 156]
ages = ["< 1yo", "1-2yo", "2-3yo", "3-4yo", "4-5yo", "5-6yo", "6-7yo", 
         "7-8yo", "8-9yo", "9-10yo", "10-11yo", "11-12yo", "12-13yo"]

dog_age_bin["age_range"] = pd.cut(dog_age_bin["ageinmonths"], bins, labels=ages, include_lowest=True)
dog_age_bin = dog_age_bin.groupby("age_range").nunique()
dog_age_bin = dog_age_bin[["petid"]]
dog_age_bin = dog_age_bin.rename(columns={"petid": "number_of_dogs"})
dog_age_bin = dog_age_bin.reset_index()
dog_age_bin

C:\Users\sakis\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,age_range,number_of_dogs
0,< 1yo,1517
1,1-2yo,228
2,2-3yo,141
3,3-4yo,75
4,4-5yo,54
5,5-6yo,29
6,6-7yo,27
7,7-8yo,17
8,8-9yo,4
9,9-10yo,6


In [14]:
dog_breed = dogs.groupby("mainbreed").nunique()
dog_breed = dog_breed[["petid"]]
dog_breed = dog_breed.rename(columns={"petid": "number_of_dogs"})
dog_breed = dog_breed.sort_values("number_of_dogs", ascending=False)
dog_breed = dog_breed.reset_index()
dog_breed

,mainbreed,number_of_dogs
0,Mixed Breed,1497
1,Shih Tzu,63
2,Labrador Retriever,54
3,Poodle,53
4,Golden Retriever,45
...,...,...
68,Cairn Terrier,1
69,Black and Tan Coonhound,1
70,Bearded Collie,1
71,Australian Shepherd,1


In [15]:
cat_health = cats.groupby(["gender", "health"]).nunique()
cat_health = cat_health[["petid"]]
cat_health = cat_health.rename(columns={"petid": "number_of_cats"})
cat_health1 = cat_health.reset_index()
cat_health

number_of_cats
gender        health                      
Female        Healthy                  803
              Major Injury               2
              Minor Injury              24
Male          Healthy                  651
              Major Injury               2
              Minor Injury              28
Not Specified Healthy                  355
              Minor Injury               7

In [16]:
cat_age = cats.groupby("adoptionspeed").mean()
cat_age = cat_age.reset_index()
cat_age["adoptionspeed"]= cat_age["adoptionspeed"].replace({0:'Same Day',1:'Within 1 Week',2:'Within 1 Month',3:'Within 2-3 Months',4:'Not Adopted, Probably Euthanized'})
cat_age

,adoptionspeed,ageinmonths
0,"Not Adopted, Probably Euthanized",15.529002
1,Same Day,2.166667
2,Within 1 Month,5.662252
3,Within 1 Week,3.679654
4,Within 2-3 Months,10.282540


In [17]:
cat_age_bin = cats[["petid","ageinmonths"]]
bins = [0, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 132, 144, 156]
ages = ["< 1yo", "1-2yo", "2-3yo", "3-4yo", "4-5yo", "5-6yo", "6-7yo", 
         "7-8yo", "8-9yo", "9-10yo", "10-11yo", "11-12yo", "12-13yo"]

cat_age_bin["age_range"] = pd.cut(cat_age_bin["ageinmonths"], bins, labels=ages, include_lowest=True)
cat_age_bin = cat_age_bin.groupby("age_range").nunique()
cat_age_bin = cat_age_bin[["petid"]]
cat_age_bin = cat_age_bin.rename(columns={"petid": "number_of_cats"})
cat_age_bin = cat_age_bin.reset_index()
cat_age_bin

C:\Users\sakis\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,age_range,number_of_cats
0,< 1yo,1590
1,1-2yo,158
2,2-3yo,66
3,3-4yo,32
4,4-5yo,8
5,5-6yo,11
6,6-7yo,1
7,7-8yo,5
8,8-9yo,0
9,9-10yo,1


In [18]:
cat_breed = cats.groupby("mainbreed").nunique()
cat_breed = cat_breed[["petid"]]
cat_breed = cat_breed.rename(columns={"petid": "number_of_cats"})
cat_breed = cat_breed.sort_values("number_of_cats", ascending=False)
cat_breed = cat_breed.reset_index()
cat_breed

,mainbreed,number_of_cats
0,Domestic Short Hair,1026
1,Domestic Medium Hair,365
2,Persian,78
3,Domestic Long Hair,71
4,Tabby,63
5,Siamese,47
6,American Shorthair,33
7,Calico,25
8,Oriental Short Hair,24
9,Bengal,14


### Connect to local database

In [19]:
rds_connection_string = "admin2:12345@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [33]:
engine.table_names()

['combined_data',
 'dog_data',
 'cat_data',
 'dog_health',
 'cat_health',
 'dog_age',
 'cat_age',
 'dog_age_bin',
 'cat_age_bin',
 'dog_breed',
 'cat_breed']

### Use pandas to load csv converted DataFrame into database

In [21]:
combined_data.to_sql(name='combined_data', con=engine, if_exists='append', index=False)
dogs.to_sql(name='dog_data', con=engine, if_exists='append', index=False)
cats.to_sql(name='cat_data', con=engine, if_exists='append', index=False)
dog_health1.to_sql(name='dog_health', con=engine, if_exists='append', index=False)
cat_health1.to_sql(name='cat_health', con=engine, if_exists='append', index=False)
dog_age.to_sql(name='dog_age', con=engine, if_exists='append', index=False)
cat_age.to_sql(name='cat_age', con=engine, if_exists='append', index=False)
dog_age_bin.to_sql(name='dog_age_bin', con=engine, if_exists='append', index=False)
cat_age_bin.to_sql(name='cat_age_bin', con=engine, if_exists='append', index=False)
dog_breed.to_sql(name='dog_breed', con=engine, if_exists='append', index=False)
cat_breed.to_sql(name='cat_breed', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying tables

In [22]:
pd.read_sql_query('select * from combined_data', con=engine).head()

,petid,adoptionspeed,type,ageinmonths,gender,health,mainbreed
0,e2dfc2935,Within 2-3 Months,Cat,8,Male,Healthy,Domestic Short Hair
1,2191e428d,"Not Adopted, Probably Euthanized",Cat,10,Female,Healthy,Domestic Short Hair
2,0ecec0922,"Not Adopted, Probably Euthanized",Cat,7,Female,Healthy,Domestic Short Hair
3,23b3f4aaf,Within 2-3 Months,Cat,15,Female,Healthy,Domestic Short Hair
4,e46543f14,"Not Adopted, Probably Euthanized",Cat,36,Female,Healthy,Domestic Short Hair


In [23]:
pd.read_sql_query('select * from dog_data', con=engine).head()

,petid,adoptionspeed,type,ageinmonths,gender,health,mainbreed
0,e02abc8a3,"Not Adopted, Probably Euthanized",Dog,12,Female,Healthy,Mixed Breed
1,0113cedff,"Not Adopted, Probably Euthanized",Dog,24,Female,Healthy,Mixed Breed
2,0070b950a,"Not Adopted, Probably Euthanized",Dog,12,Female,Healthy,Mixed Breed
3,be85036be,"Not Adopted, Probably Euthanized",Dog,5,Male,Healthy,Mixed Breed
4,6a968e033,"Not Adopted, Probably Euthanized",Dog,6,Not Specified,Healthy,Mixed Breed


In [24]:
pd.read_sql_query('select * from cat_data', con=engine).head()

,petid,adoptionspeed,type,ageinmonths,gender,health,mainbreed
0,e2dfc2935,Within 2-3 Months,Cat,8,Male,Healthy,Domestic Short Hair
1,2191e428d,"Not Adopted, Probably Euthanized",Cat,10,Female,Healthy,Domestic Short Hair
2,0ecec0922,"Not Adopted, Probably Euthanized",Cat,7,Female,Healthy,Domestic Short Hair
3,23b3f4aaf,Within 2-3 Months,Cat,15,Female,Healthy,Domestic Short Hair
4,e46543f14,"Not Adopted, Probably Euthanized",Cat,36,Female,Healthy,Domestic Short Hair


In [25]:
pd.read_sql_query('select * from dog_health', con=engine).head()

,gender,health,number_of_dogs
0,Female,Healthy,1011
1,Female,Major Injury,2
2,Female,Minor Injury,36
3,Male,Healthy,780
4,Male,Major Injury,3


In [26]:
pd.read_sql_query('select * from cat_health', con=engine).head()

,gender,health,number_of_cats
0,Female,Healthy,803
1,Female,Major Injury,2
2,Female,Minor Injury,24
3,Male,Healthy,651
4,Male,Major Injury,2


In [27]:
pd.read_sql_query('select * from dog_age', con=engine).head()

,adoptionspeed,ageinmonths
0,"Not Adopted, Probably Euthanized",16.800752
1,Same Day,14.040816
2,Within 1 Month,10.813187
3,Within 1 Week,16.563953
4,Within 2-3 Months,10.928375


In [28]:
pd.read_sql_query('select * from cat_age', con=engine).head()

,adoptionspeed,ageinmonths
0,"Not Adopted, Probably Euthanized",15.529002
1,Same Day,2.166667
2,Within 1 Month,5.662252
3,Within 1 Week,3.679654
4,Within 2-3 Months,10.282540


In [29]:
pd.read_sql_query('select * from dog_age_bin', con=engine).head()

,age_range,number_of_dogs
0,< 1yo,1517
1,1-2yo,228
2,2-3yo,141
3,3-4yo,75
4,4-5yo,54


In [30]:
pd.read_sql_query('select * from cat_age_bin', con=engine).head()

,age_range,number_of_cats
0,< 1yo,1590
1,1-2yo,158
2,2-3yo,66
3,3-4yo,32
4,4-5yo,8


In [31]:
pd.read_sql_query('select * from dog_breed', con=engine).head()

,mainbreed,number_of_dogs
0,Mixed Breed,1497
1,Shih Tzu,63
2,Labrador Retriever,54
3,Poodle,53
4,Golden Retriever,45


In [32]:
pd.read_sql_query('select * from cat_breed', con=engine).head()

,mainbreed,number_of_cats
0,Domestic Short Hair,1026
1,Domestic Medium Hair,365
2,Persian,78
3,Domestic Long Hair,71
4,Tabby,63
